In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [7]:
# =============================================================
# Pure XGBoost lap‑time predictor (single split, early stopping)
# =============================================================
!pip install --quiet xgboost==2.0.3 pandas==2.2.2

import pandas as pd, numpy as np, os, xgboost as xgb, time, gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# ── 1.  Paths & column names ─────────────────────────────────
DATA_DIR = "/kaggle/input/burnout-datathon-ieeecsmuj"   
TARGET   = "Lap_Time_Seconds"                # ⬅️ lap‑time column
ID_COL   = "Unique ID"                       # ⬅️ ID column for submission

train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
print("Train shape:", train_df.shape, "  Test shape:", test_df.shape)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 90.0 MB/s eta 0:00:00:00:01:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.1 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0de

In [8]:
# ── 2.  Minimal feature engineering (checks each column) ────
def add_features(df):
    if {"Circuit_Length_km", "Avg_Speed_kmh"}.issubset(df.columns):
        df["secs_per_km"] = (df["Circuit_Length_km"] / df["Avg_Speed_kmh"]) * 3600
    if {"Track_Temperature_Celsius", "Ambient_Temperature_Celsius"}.issubset(df.columns):
        df["track_minus_air"] = df["Track_Temperature_Celsius"] - df["Ambient_Temperature_Celsius"]
    if "Track_Condition" in df.columns:
        df["is_wet"] = df["Track_Condition"].str.contains("Wet", case=False, na=False).astype(int)
    if {"Tire_Compound_Front", "Tire_Compound_Rear"}.issubset(df.columns):
        df["tire_combo"] = df["Tire_Compound_Front"].fillna("") + "_" + df["Tire_Compound_Rear"].fillna("")
    return df

train_df = add_features(train_df);  test_df = add_features(test_df)


In [9]:
# ── 3.  Convert position numeric → categorical string (if present) ──
if "position" in train_df.columns:
    train_df["Position_cat"] = train_df["position"].astype(str)
    test_df["Position_cat"]  = test_df["position"].astype(str)
    train_df.drop(columns=["position"], inplace=True)
    test_df.drop(columns=["position"],  inplace=True)

In [10]:
# ── 4.  Identify categorical & numeric columns robustly ──────
cat_cols = [c for c in train_df.columns
            if train_df[c].dtype == "object" or train_df[c].dtype.name == "category"]
num_cols = [c for c in train_df.columns if c not in cat_cols + [TARGET]]

# Keep only cats present in both frames
cat_cols = [c for c in cat_cols if c in test_df.columns]

# Fill NaNs
train_df[cat_cols] = train_df[cat_cols].fillna("missing")
test_df[cat_cols]  = test_df[cat_cols].fillna("missing")
for col in num_cols:
    med = train_df[col].median()
    train_df[col].fillna(med, inplace=True)
    test_df[col].fillna(med,  inplace=True)

# Cast categorical dtype for XGB native cats
for c in cat_cols:
    train_df[c] = train_df[c].astype("category")
    test_df[c]  = test_df[c].astype("category")

feature_cols = cat_cols + num_cols
print(f"Using {len(feature_cols)} features ({len(cat_cols)} categorical).")

/tmp/ipykernel_35/41244590.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(med, inplace=True)
/tmp/ipykernel_35/41244590.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

Using 48 features (15 categorical).


In [11]:
# ── 5.  Train‑validation split ───────────────────────────────
X_train, X_val, y_train, y_val = train_test_split(
    train_df[feature_cols], train_df[TARGET],
    test_size=0.2, random_state=42
)

dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dval   = xgb.DMatrix(X_val,   label=y_val,   enable_categorical=True)
dtest  = xgb.DMatrix(test_df[feature_cols], enable_categorical=True)

In [12]:
# ── 6.  XGBoost parameters & training ────────────────────────
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "learning_rate": 0.05,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 10,
    "lambda": 3.0,
    "tree_method": "hist",          # ➡ “gpu_hist” if you enabled GPU runtime
    "enable_categorical": True,
    "seed": 42,
}

start = time.time()
model = xgb.train(
    params,
    dtrain,
    num_boost_round=5000,
    evals=[(dval, "val")],
    early_stopping_rounds=200,
    verbose_eval=250
)
print(f"⏱️  Training time: {time.time()-start:.1f} s")
print("🔎 Best iteration:", model.best_iteration)
print("🏁 Best val RMSE:", model.best_score)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [08:49:41] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "enable_categorical" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	val-rmse:11.53022
[250]	val-rmse:9.30236
[500]	val-rmse:6.57608
[750]	val-rmse:4.48289
[1000]	val-rmse:3.07405
[1250]	val-rmse:2.20035
[1500]	val-rmse:1.54698
[1750]	val-rmse:1.09824
[2000]	val-rmse:0.77363
[2250]	val-rmse:0.55098
[2500]	val-rmse:0.39761
[2750]	val-rmse:0.28674
[3000]	val-rmse:0.21577
[3250]	val-rmse:0.16657
[3500]	val-rmse:0.13309
[3750]	val-rmse:0.10816
[4000]	val-rmse:0.09129
[4250]	val-rmse:0.07856
[4500]	val-rmse:0.06928
[4750]	val-rmse:0.06222
[4999]	val-rmse:0.05681
⏱️  Training time: 2055.7 s
🔎 Best iteration: 4999
🏁 Best val RMSE: 0.05681306002656589


In [13]:
# ── 7.  Predict on test & create submission ──────────────────
test_preds = model.predict(dtest, iteration_range=(0, model.best_iteration + 1))
submission = pd.DataFrame({ID_COL: test_df[ID_COL], TARGET: test_preds})
submission.to_csv("submission_xgb.csv", index=False)
print("✅ submission_xgb.csv saved:", submission.shape)
submission.head()

✅ submission_xgb.csv saved: (546874, 2)


,Unique ID,Lap_Time_Seconds
0,288307,89.853821
1,704288,104.082764
2,951491,86.450500
3,2591721,109.790520
4,1202653,99.353638
